In [1]:
## imports
# astronomy
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
# plotting
import matplotlib.pyplot as plt
# data 
import numpy as np
import pandas as pd 

# Read Data

In [2]:
# file paths 
path_catalogs = 'C:\\Users\\polar\\OneDrive - The University of Kansas\\AGNerds\\Catalogs'

In [3]:
# open tphot
tphot_data = ascii.read(path_catalogs+'\\tphot.cat')
tphot_cols = tphot_data.colnames

# # show table
# tphot_data.show_in_notebook()
# print(tphot_cols)

In [4]:
# open EGS F22
egs_f22_data = Table.read(path_catalogs+'\\EGS_F22.fits') # ??? something is wrong with the dimentsions... TODO just build new table manually 
egs_f22_cols = egs_f22_data.columns

# # print all columns
# for col in egs_f22_cols:
#     print(col)

In [5]:
# open EGS FIR
egs_fir_data = Table.read(path_catalogs+'\\egs_FIR_photometry_catalog.fits')
egs_fir_cols = egs_fir_data.columns

# # print all columns
# for col in egs_fir_cols:
#     print(col)

# Functions

In [6]:
def RaDec_to_Coord(ra,dec):
    return( SkyCoord(ra=ra*u.deg, dec=dec*u.deg) )

In [7]:
def GetCoordMatches(coord_cat, coord_c, separation=1.0) : 
    # match c to catalog 
    #   idx are indices into catalog that are the closest objects to each of the coordinates in c, 
    #   d2d are the on-sky distances between them, and 
    #   d3d are the 3-dimensional distances. 
    idx, d2d, d3d = coord_c.match_to_catalog_sky(coord_cat) # idx, d2d, d3d = c.match_to_catalog_sky(catalog)

    # separation constraint
    max_sep = separation * u.arcsec
    sep_constraint = d2d < max_sep  # use on 'c'
    idx_sep = idx[sep_constraint]   # use on 'catalog'

    # Get array of indeces 
    coordCat_matches = idx_sep
    coordC_matches   = np.where(sep_constraint)[0]
    
    # return matched sources
    return(coordCat_matches,coordC_matches)

In [8]:
def TestCoord(c1,c2,i):
    print('Coordinant 1: ', c1[i]) 
    print('Coordinant 2: ', c2[i])

In [9]:
def GetDuplicatesMask(arr): 
    # make mask of unique soruces 
    mask = np.zeros(len(arr), dtype=bool)
    mask[np.unique(arr, return_index=True)[1]] = True
    # get value of duplicates sources 
    duplicates = np.unique(arr[~mask])
    # set all non-unique sources to False 
    for dup in duplicates :
        mask[np.where(arr == dup)] = False
    # return boolean mask of length(arr) that is True for a value that occurs more than once
    return(mask)

# Match EGS catalogs using RA/DEC

In [10]:
# Get EGS coordinants
egs_f22_coord = RaDec_to_Coord(np.array(egs_f22_data['RA'])[0], np.array(egs_f22_data['DEC'])[0])
egs_fir_coord = RaDec_to_Coord(np.array(egs_fir_data['RA']),    np.array(egs_fir_data['Decl'])  )

# print info
print('Number of EGS f22 sources:\t', len(egs_f22_coord))
print('Number of EGS fir sources:\t', len(egs_fir_coord))

Number of EGS f22 sources:	 66783
Number of EGS fir sources:	 41656


In [11]:
# match 
egs_f22_toFIR_match, egs_fir_toF22_match = GetCoordMatches(egs_f22_coord,egs_fir_coord)

# get matches
egs_f22_coord_matches = egs_f22_coord[egs_f22_toFIR_match]
egs_fir_coord_matches = egs_fir_coord[egs_fir_toF22_match]

# print info 
print('Number of egsF22 matches:\t',  len(egs_f22_coord_matches))
print('Number of egsFIR matches:\t',  len(egs_fir_coord_matches))

Number of egsF22 matches:	 40850
Number of egsFIR matches:	 40850


In [12]:
TestCoord(egs_f22_coord_matches,egs_fir_coord_matches,1)

Coordinant 1:  <SkyCoord (ICRS): (ra, dec) in deg
    (215.267114, 53.027554)>
Coordinant 2:  <SkyCoord (ICRS): (ra, dec) in deg
    (215.26714, 53.02755)>


# Get Unique sources from EGS

In [13]:
# get mask of egs objects that occur more than once 
egs_duplicatesMask = GetDuplicatesMask(egs_f22_toFIR_match)

In [14]:
# apply mask to get unique and duplicate sources
egs_f22_unique_i     = egs_f22_toFIR_match[egs_duplicatesMask]
egs_f22_duplicates_i = egs_f22_toFIR_match[~egs_duplicatesMask]
# apply mask to get unique and duplicate sources
egs_fir_unique_i     = egs_fir_toF22_match[egs_duplicatesMask]
egs_fir_duplicates_i = egs_fir_toF22_match[~egs_duplicatesMask]

In [15]:
# print info
print('Number of unique:\t', len(egs_f22_unique_i))
print('Number of duplicates:\t', len(egs_f22_duplicates_i))

Number of unique:	 40571
Number of duplicates:	 279


# Match tphot to EGS f22 using RA/DEC

In [16]:
# get coordinants
tphot_coord = RaDec_to_Coord(np.array(tphot_data['ra']), np.array(tphot_data['dec']) )
egs_coord   = RaDec_to_Coord(np.array(egs_f22_data['RA'] )[0][egs_f22_unique_i], np.array(egs_f22_data['DEC'])[0][egs_f22_unique_i])

# print info 
print('Number of tphot sources:\t', len(tphot_coord))
print('Number of egs sources:\t\t',   len(egs_coord))

Number of tphot sources:	 1734
Number of egs sources:		 40571


In [17]:
# match 
tphot_toEGS_match, egs_toTphot_match = GetCoordMatches(tphot_coord,egs_coord)

# get matches
tphot_coord_matches = tphot_coord[tphot_toEGS_match]
egs_coord_matches = egs_coord[egs_toTphot_match]

# print length  
print('Number of tphot matches:\t', len(tphot_coord_matches))
print('Number of EGS matches:\t\t', len(egs_coord_matches))

Number of tphot matches:	 1655
Number of EGS matches:		 1655


In [18]:
TestCoord(tphot_coord_matches,egs_coord_matches,1)

Coordinant 1:  <SkyCoord (ICRS): (ra, dec) in deg
    (215.061606, 52.901508)>
Coordinant 2:  <SkyCoord (ICRS): (ra, dec) in deg
    (215.061586, 52.901517)>


# Find Duplicate RA/DEC Matches

In [19]:
# get mask of tphot objects that occur more than once 
tphot_duplicateMask = GetDuplicatesMask(tphot_toEGS_match)

In [20]:
# apply mask to get unique and duplicate sources
tphot_unique_i     = tphot_toEGS_match[tphot_duplicateMask]
tphot_duplicates_i = tphot_toEGS_match[~tphot_duplicateMask]
# apply mask to get unique and duplicate sources
egs_unique_i       = egs_toTphot_match[tphot_duplicateMask]
egs_duplicates_i   = egs_toTphot_match[~tphot_duplicateMask]

In [21]:
# print info
print('Number of unique:\t', len(egs_unique_i))
print('Number of duplicates:\t', len(egs_duplicates_i))

Number of unique:	 1608
Number of duplicates:	 47


In [22]:
# get coord values 
tphot_coord_unique = tphot_coord[tphot_unique_i]
egs_coord_unique = egs_coord[egs_unique_i]
TestCoord(tphot_coord_unique,egs_coord_unique,1)

Coordinant 1:  <SkyCoord (ICRS): (ra, dec) in deg
    (215.061606, 52.901508)>
Coordinant 2:  <SkyCoord (ICRS): (ra, dec) in deg
    (215.061586, 52.901517)>


# Build Catalog

In [26]:
# helper indexing
tphot_i     = tphot_unique_i
egs_f22_i   = egs_f22_unique_i[egs_unique_i] 
egs_fir_i   = egs_fir_unique_i[egs_unique_i]

# verify that all lengths match
print(len(tphot_i))
print(len(egs_f22_i))
print(len(egs_fir_i))

# save number of matches 
n_matches = len(tphot_i)

1608
1608
1608


In [25]:
# # convert astropy table to pandas dataframe
# tphot_df   = tphot_data.to_pandas()
# egs_f22_df = egs_f22_data.to_pandas() # ??? busted... TODO build table by processing columns individually 
# egs_fir_df = egs_fir_data.to_pandas()

# # setup table with column names 
# # populate table row by row
